In [1]:
# CÓDIGO PARA: 1_calibracion.ipynb

import sys
import os
import numpy as np
import time

# --- 1. CONFIGURACIÓN DE RUTAS ---
# Esto es necesario para que Python encuentre la carpeta 'modelo'
notebook_dir = os.path.abspath(os.path.dirname(''))
project_root = os.path.dirname(notebook_dir)
if project_root not in sys.path:
    sys.path.append(project_root)

# Importamos tu modelo desde el archivo local
from modelo_ESPAÑA.modelo import ModeloAnisi

print("--- CALIBRACIÓN AUTOMÁTICA: ESPAÑA 2024 (Datos Reales) ---")

# --- 2. PARÁMETROS DE BÚSQUEDA ---
POBLACION = 1000
NUM_ITERACIONES = 400  # Cuantas más mejor

# RANGOS DE BÚSQUEDA (Ajustados a la economía de 2024)
# Con productividad z=68 y salarios w=17.5, el gasto necesario es alto.
RANGO_GA = [35000, 55000]  # Gasto Público
RANGO_C  = [0.4, 0.85]     # Propensión al consumo
RANGO_ZB = [5.0, 25.0]     # Productividad doméstica

# --- 3. FUNCIÓN OBJETIVO ---
def calcular_error(modelo):
    # OBJETIVOS REALES (INE / OECD 2024)
    TARGET_PARO = 0.114  # 11.4%
    TARGET_I3 = 1.10     # Tensión de consumo moderada (típica país desarrollado)
    
    # Resultados de la simulación
    tasa_paro_sim = 1.0 - (modelo.empleo_total / modelo.N)
    i3_sim = modelo.I3_agregado
    
    # Penalizaciones (Evitar resultados matemáticamente rotos)
    if i3_sim > 3.0 or i3_sim < 0 or np.isnan(i3_sim): return 1000.0
    
    # Calculamos el error (Damos mucha prioridad a clavar el Paro)
    error = (abs(tasa_paro_sim - TARGET_PARO) * 300) + (abs(i3_sim - TARGET_I3) * 50)
    return error

# --- 4. OPTIMIZACIÓN MONTE CARLO ---
mejor_error = float('inf')
mejores_params = None

print(f"Buscando equilibrio (Objetivo: Paro 11.4%) en {NUM_ITERACIONES} iteraciones...")
start_time = time.time()

for i in range(NUM_ITERACIONES):
    # Generar hipótesis aleatoria
    g_val = np.random.uniform(*RANGO_GA)
    c_val = np.random.uniform(*RANGO_C)
    z_val = np.random.uniform(*RANGO_ZB)
    
    try:
        # Instanciamos el modelo con DATOS 2024
        modelo = ModeloAnisi(
            # Datos Fijos Reales
            t=0.30, j=1.0,           # Presión fiscal 30%
            z_media=68.0, z_std=5.0, # PIB/hora ~68€
            w_media=17.5, w_std=2.0, # Coste laboral neto/hora ~17.5€
            # Hipótesis a probar
            G_a=g_val,
            c_media=c_val, c_std=0.05, 
            zb_media=z_val, zb_std=3.0,
            N_trabajadores=POBLACION, N_empresas=50, T_i_total=2000
        )
        modelo.step()
        err = calcular_error(modelo)
        
        # ¿Es mejor que lo anterior?
        if err < mejor_error:
            mejor_error = err
            mejores_params = [g_val, c_val, z_val]
            
            # Feedback visual
            paro_actual = (1 - modelo.empleo_total/POBLACION)*100
            print(f" -> ¡Mejora! (Iter {i}) | Error: {err:.2f} | Paro: {paro_actual:.1f}% | I3: {modelo.I3_agregado:.3f}")
            
    except Exception: continue

# --- 5. RESULTADOS FINALES ---
print("\n" + "="*40)
if mejores_params:
    print("   ¡CALIBRACIÓN FINALIZADA!")
    print("="*40)
    print(f"Mejor Error: {mejor_error:.4f}")
    print("PARÁMETROS ÓPTIMOS (Copia estos valores para el siguiente notebook):")
    print(f"  GA_OPTIMO = {mejores_params[0]:.2f}")
    print(f"  C_OPTIMO  = {mejores_params[1]:.4f}")
    print(f"  ZB_OPTIMO = {mejores_params[2]:.2f}")
    
    # Validación Final (Media de 5 ejecuciones)
    print("\n--- VALIDACIÓN DE ROBUSTEZ (Promedio 5 tests) ---")
    paros = []
    for k in range(5):
        m = ModeloAnisi(
            t=0.30, j=1.0, z_media=68.0, z_std=5.0, w_media=17.5, w_std=2.0,
            G_a=mejores_params[0],
            c_media=mejores_params[1], c_std=0.05,
            zb_media=mejores_params[2], zb_std=3.0,
            N_trabajadores=POBLACION, N_empresas=50, T_i_total=2000
        )
        m.step()
        paros.append((1 - m.empleo_total/POBLACION)*100)
    
    print(f"Tasa de Paro Media Resultante: {sum(paros)/len(paros):.2f}%")
else:
    print("❌ No se encontró una solución estable. Intenta ampliar los rangos.")

Could not import SolaraViz. If you need it, install with 'pip install --pre mesa[viz]'
--- CALIBRACIÓN AUTOMÁTICA: ESPAÑA 2024 (Datos Reales) ---
Buscando equilibrio (Objetivo: Paro 11.4%) en 400 iteraciones...
 -> ¡Mejora! (Iter 0) | Error: 70.01 | Paro: 20.7% | I3: 0.258
 -> ¡Mejora! (Iter 1) | Error: 58.33 | Paro: 0.0% | I3: 0.617
 -> ¡Mejora! (Iter 5) | Error: 38.60 | Paro: 8.0% | I3: 0.532
 -> ¡Mejora! (Iter 12) | Error: 33.17 | Paro: 13.4% | I3: 0.557
 -> ¡Mejora! (Iter 41) | Error: 27.82 | Paro: 10.5% | I3: 0.598
 -> ¡Mejora! (Iter 55) | Error: 24.17 | Paro: 10.8% | I3: 0.653
 -> ¡Mejora! (Iter 91) | Error: 16.76 | Paro: 7.2% | I3: 1.183

   ¡CALIBRACIÓN FINALIZADA!
Mejor Error: 16.7585
PARÁMETROS ÓPTIMOS (Copia estos valores para el siguiente notebook):
  GA_OPTIMO = 44897.45
  C_OPTIMO  = 0.6491
  ZB_OPTIMO = 5.84

--- VALIDACIÓN DE ROBUSTEZ (Promedio 5 tests) ---
Tasa de Paro Media Resultante: 11.70%
